In [1]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import pickle

# Load the pre-trained XGBoost model
with open("hsc_optimisers_normal_bboa_gbm_model.pkl", "rb") as file:
    final_model = pickle.load(file)

# Load the Excel file and get feature names
excel_file = "SCC.xlsx"
df = pd.read_excel(excel_file, sheet_name="Sheet1")
features = list(df.columns[:-1])  # Assuming the last column is the target variable

# Create tkinter window
root = tk.Tk()
root.title("Predict Using XGBoost Model")

# Main frame to hold the scrollable content
main_frame = tk.Frame(root)
main_frame.pack(fill=tk.BOTH, expand=1)

# Create a canvas in the main frame
canvas = tk.Canvas(main_frame)
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)

# Add a scrollbar to the canvas
scrollbar = ttk.Scrollbar(main_frame, orient=tk.VERTICAL, command=canvas.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Configure canvas with scrollbar
canvas.configure(yscrollcommand=scrollbar.set)
canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# Create another frame inside the canvas
input_frame = tk.Frame(canvas)
canvas.create_window((0, 0), window=input_frame, anchor="nw")

# Styling for the input fields
entry_style = {
    'font': ('Futura Md Bt', 12),
    'bg': '#f5f5f5',
    'relief': tk.GROOVE
}

label_style = {
    'font': ('Futura Md Bt', 12),
    'bg': '#e9ecef',
    'anchor': 'w'
}

# Create a dictionary to hold input entries
entries = {}

# Add input fields for each feature dynamically
for i, feature in enumerate(features):
    label = tk.Label(input_frame, text=f"{feature}:", **label_style)
    label.grid(row=i, column=0, sticky="w", padx=10, pady=5)
    
    entry = tk.Entry(input_frame, **entry_style)
    entry.grid(row=i, column=1, padx=10, pady=5)
    
    entries[feature] = entry

# Function to handle the prediction
def predict():
    input_data = []
    
    # Gather data from input fields
    for feature, entry in entries.items():
        value = entry.get().strip()
        if value:
            try:
                input_data.append(float(value))  # Convert input to float
            except ValueError:
                messagebox.showerror("Error", f"Invalid input for {feature}. Please enter a valid number.")
                return
        else:
            messagebox.showerror("Error", f"Please fill in all fields for {feature}.")
            return

    # Convert to DataFrame for the model
    input_df = pd.DataFrame([input_data], columns=features)
    
    # Predict using the loaded model
    prediction = final_model.predict(input_df)
    
    # Display prediction result
    result_label.config(text=f"Predicted Value: {prediction[0]:.2f}")

# Add the Predict button at the end
predict_button = tk.Button(input_frame, text="Predict", command=predict, bg='#007bff', fg='white', font=('Futura Md Bt', 12), padx=10, pady=5)
predict_button.grid(row=len(features), column=1, pady=20)

# Add a label to display the prediction result
result_label = tk.Label(input_frame, text="", font=('Futura Md Bt', 14), bg='#e9ecef', fg='#28a745')
result_label.grid(row=len(features)+1, column=0, columnspan=2, pady=20)

# Set a minimum window size
root.geometry("600x500")

# Run the application
root.mainloop()